In [119]:
from business_models.databases import gdocs
import pandas as pd

In [116]:
YNDX_GEOCODER_API_KEY='XXXXXXX'

In [117]:
sheet_id = '1SSHy89eh0h8MIQ-fEpbr_oKdtbBAlQyaKC4tfnV__vM'

data = df = gdocs.read(
    table_name='Addresses Chile Santiago',  # Название листа в google.sheet
    sheet_id=sheet_id,   # ID google.sheet (берется из ссылки)
    header=1,            # Количество строк-заголовков
#     index=1            # Количество столбцов индексов. Если не передать, индекса не будет
).to_dict(orient='records')

In [4]:
len(data)

1772

In [5]:
data[0]

{'Address': 'Forestal 2930 Conchalí 2930, CONCHALI, Santiago', 'Found': ''}

In [66]:
from math import sin, cos, sqrt, atan2, radians

def points_distance(lat1, lon1, lat2, lon2): # возвращает километры
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

assert abs(points_distance(-33.519082, -70.653037, -33.519060, -70.651811) - 0.1) < 0.2

In [114]:
import requests
session = requests.Session()

def coords_by_query(query): # returns lat, lon
    yndx_url = 'https://geocode-maps.yandex.ru/1.x/?geocode={0}&format=json&apikey={1}'.format(query, YNDX_GEOCODER_API_KEY)

    resp = session.get(yndx_url)
    try:
        obj = resp.json()['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')
    except Exception as ex:
        print(query, ex)
        return None, None
    
    return float(obj[1]), float(obj[0])

In [11]:
coords_by_query('San Vicente 981, Santiago, Región Metropolitana, Чили')

(-33.46149, -70.676437)

In [63]:
def is_tlbr_match(lat, lon):
    if lat is None or lon is None:
        return False
    
    top_left_santiago = -33.273324, -70.851584 # lat, lon
    bottom_right_santiago = -33.677120, -70.429602
    
    return (bottom_right_santiago[0] <= lat <= top_left_santiago[0]) and \
           (top_left_santiago[1] <= lon <= bottom_right_santiago[1])

In [13]:
assert is_tlbr_match(-33.461495, -70.639175)
assert not is_tlbr_match(-33.103287, -71.317399)
assert not is_tlbr_match(None, -1)

In [ ]:
for i in range(len(data)):
    if i % 100 == 1:
        print(i)

    data[i]['yndx_lat'], data[i]['yndx_lon'] = coords_by_query(data[i]['Address'])

In [15]:
for i in range(len(data)):
    data[i]['has_yndx_response'] = int(data[i]['yndx_lat'] is not None)
    data[i]['yndx_response_tlbr_match'] = int(is_tlbr_match(data[i]['yndx_lat'], data[i]['yndx_lon']))

In [16]:
data[0]

{'Address': 'Forestal 2930 Conchalí 2930, CONCHALI, Santiago',
 'Found': '',
 'yndx_lat': -33.375793,
 'yndx_lon': -70.672287,
 'has_yndx_response': 1,
 'yndx_response_tlbr_match': 1}

In [20]:
df = pd.DataFrame(data)
df['has_yndx_response'].mean(), df['yndx_response_tlbr_match'].mean(), df[df['has_yndx_response'] == 1]['yndx_response_tlbr_match'].mean()

(0.9390519187358917, 0.9260722347629797, 0.9861778846153846)

In [21]:
df[df['has_yndx_response'] == 1]

Address Found   yndx_lat  \
0       Forestal 2930 Conchalí 2930, CONCHALI, Santiago       -33.375793   
2            Pasaje La Fuente 0464, QUILICURA, Santiago       -33.382267   
3                 EL CLAUSTRO 0516, QUILICURA, Santiago       -33.363492   
4                    Lo ovalle 410, QUILICURA, Santiago       -33.508145   
5     Avenida Las Torres Norte 192, QUILICURA, Santiago       -33.382267   
...                                                 ...   ...        ...   
1766        tucan 511, maipu casa roja, MAIPU, Santiago       -33.507076   
1767  Diagonal Arqto Gonzalo Mardones Restat 436, MA...       -33.430599   
1768                 el estanque 51034, MAIPU, Santiago       -33.507129   
1769          Pasaje Los Zorzales 1138, MAIPU, Santiago       -33.357258   
1770       Francisco Coloane 18600, Maipú, Maipu, Chile       -33.519138   

       yndx_lon  has_yndx_response  yndx_response_tlbr_match  
0    -70.672287                  1                         1  
2    -70.685223                  1                         1  
3    -70.705794                  1                         1  
4    -70.676410                  1                         1  
5    -70.685223                  1                         1  
...         ...                ...                       ...  
1766 -70.752192                  1                         1  
1767 -70.620391                  1                         1  
1768 -70.769332                  1                         1  
1769 -70.690379                  1                         1  
1770 -70.739768                  1                         1  

[1664 rows x 6 columns]

In [32]:
df.groupby(['has_yndx_response', 'yndx_response_tlbr_match']).size() / 1772

has_yndx_response  yndx_response_tlbr_match
0                  0                           0.060948
1                  0                           0.012980
                   1                           0.926072
dtype: float64

In [23]:
sum(df['has_yndx_response'] == df['yndx_response_tlbr_match']), len(df['has_yndx_response'] == df['yndx_response_tlbr_match'])

(1749, 1772)

In [25]:
df.to_excel('LOGDATA-1133.01.xlsx', index=None)

In [ ]:
# не найденные адреса

for addr in df[df['has_yndx_response'] == 0].reindex().head(50)['Address']:
    print(addr)

In [6]:
# восстанавливаемся из эксельки после перезапущенного ядра

df = pd.read_excel('LOGDATA-1133.01.xlsx', engine='openpyxl',)
df.head()

Address  Found   yndx_lat  \
0    Forestal 2930 Conchalí 2930, CONCHALI, Santiago    NaN -33.375793   
1  piedra roja 123 casa 100 100, QUILICURA, Santiago    NaN        NaN   
2         Pasaje La Fuente 0464, QUILICURA, Santiago    NaN -33.382267   
3              EL CLAUSTRO 0516, QUILICURA, Santiago    NaN -33.363492   
4                 Lo ovalle 410, QUILICURA, Santiago    NaN -33.508145   

    yndx_lon  has_yndx_response  yndx_response_tlbr_match  
0 -70.672287                  1                         1  
1        NaN                  0                         0  
2 -70.685223                  1                         1  
3 -70.705794                  1                         1  
4 -70.676410                  1                         1

In [7]:
data = df.to_dict(orient='records')

In [9]:
data[0]

{'Address': 'Forestal 2930 Conchalí 2930, CONCHALI, Santiago',
 'Found': nan,
 'yndx_lat': -33.375793,
 'yndx_lon': -70.672287,
 'has_yndx_response': 1,
 'yndx_response_tlbr_match': 1}

In [11]:
import geocoder # pip install geocoder, https://geocoder.readthedocs.io/

In [52]:
import time

In [59]:
for i in range(len(data)):
    if i % 100 == 1:
        print(i)

    g = geocoder.google(data[i]['Address'], method='geocode', key='GOOGLE_API_KEY')
    if g:
        data[i]['google_lat'], data[i]['google_lon'] = g.latlng
    else:
        data[i]['google_lat'], data[i]['google_lon'] = None, None
        
    data[i]['has_google_response'] = data[i]['google_lat'] is not None
    
    time.sleep(0.2)
    

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701


In [61]:
for i in range(len(data)):
    data[i]['has_google_response'] = int(data[i]['has_google_response'])

df = pd.DataFrame(data)

df.to_excel('LOGDATA-1133.02.xlsx', index=None)

In [ ]:
for i in range(len(data)):
    data[i]['google_response_tlbr_match'] = int(is_tlbr_match(data[i]['google_lat'], data[i]['google_lon']))
    data[i]['google_yndx_diff_meters'] = 1000 * points_distance(data[i]['google_lat'], data[i]['google_lon'], 
                                                             data[i]['yndx_lat'], data[i]['yndx_lon'])

df = pd.DataFrame(data)
df.head()

In [83]:
df['has_yndx_response'].mean(), df['has_google_response'].mean()

(0.9390519187358917, 1.0)

In [109]:
for diff_threshold in [10, 20, 50, 100]:
    ddf = df[(df['has_yndx_response'] == 1)]
    # print((df['google_yndx_diff_meters'] > diff_threshold).mean())
    print(diff_threshold, '%0.2f' % (100 * (ddf['google_yndx_diff_meters'] > diff_threshold).mean()))
    # print()

10 90.81
20 82.57
50 73.62
100 69.65


In [110]:
ddf.groupby(['yndx_lat', 'yndx_lon']).size().nlargest(15)

yndx_lat    yndx_lon  
-33.396871  -70.550161    147
-33.396683  -70.567328     80
-33.484994  -70.601688     69
-33.398001  -70.662145     34
-33.507129  -70.769332     21
-33.514383  -70.685914     20
-33.443591  -70.754393     15
-33.414901  -70.603898     13
-33.503118  -70.685187     11
-33.482856  -70.744008     11
-33.431879  -70.763771     11
-33.421590  -70.601392     11
-33.382267  -70.685223     11
-33.455888  -70.690631     10
-33.444291  -70.536686      9
dtype: int64

In [111]:
ddf.groupby(['google_lat', 'google_lat']).size().nlargest(15)

google_lat  google_lat
-33.509493  -33.509493    3
-33.490940  -33.490940    3
-33.465914  -33.465914    3
-33.456851  -33.456851    3
-33.453401  -33.453401    3
-33.360727  -33.360727    3
-33.631296  -33.631296    2
-33.553503  -33.553503    2
-33.553151  -33.553151    2
-33.544010  -33.544010    2
-33.539070  -33.539070    2
-33.535245  -33.535245    2
-33.532120  -33.532120    2
-33.525137  -33.525137    2
-33.524171  -33.524171    2
dtype: int64

In [104]:
df['has_google_response'].mean()

1.0

In [105]:
df['google_response_tlbr_match'].mean()

0.9847629796839729

In [118]:
coords_by_query('Москва улица 2020 года')

(55.755819, 37.617644)

In [120]:
df = pd.read_excel('LOGDATA-1133.02.xlsx', engine='openpyxl')

In [122]:
data = df.to_dict(orient='records')

In [124]:
data[0]

{'Address': 'Forestal 2930 Conchalí 2930, CONCHALI, Santiago',
 'Found': nan,
 'yndx_lat': -33.375793,
 'yndx_lon': -70.672287,
 'has_yndx_response': 1,
 'yndx_response_tlbr_match': 1,
 'google_lat': -33.3666706,
 'google_lon': -70.6977879,
 'has_google_response': 1}

In [ ]:
import requests
import time

session = requests.Session()

for i in range(len(data)):
    if i % 100 == 5:
        print(i)

    params = {'q': data[i]['Address'], 'format': 'jsonv2'}
    resp = session.get('https://nominatim.openstreetmap.org/search.php', params=params).json()

    if len(resp) == 0:
        data[i]['has_osm_response'], data[i]['osm_lat'], data[i]['osm_lon'] = False, None, None
    else:
        data[i]['has_osm_response'], data[i]['osm_lat'], data[i]['osm_lon'] = True, resp[0]['lat'], resp[0]['lon']
          
    time.sleep(0.2)

In [131]:
data[0]

{'Address': 'Forestal 2930 Conchalí 2930, CONCHALI, Santiago',
 'Found': nan,
 'yndx_lat': -33.375793,
 'yndx_lon': -70.672287,
 'has_yndx_response': 1,
 'yndx_response_tlbr_match': 1,
 'google_lat': -33.3666706,
 'google_lon': -70.6977879,
 'has_google_response': 1,
 'osm_lat': '-33.3869456',
 'osm_lon': '-70.6740886',
 'has_osm_response': True}

'-33.3676812'

In [137]:
for i in range(len(data)):
    if data[i]['osm_lat']:
        data[i]['osm_lat'], data[i]['osm_lon'] = float(data[i]['osm_lat']), float(data[i]['osm_lon'])
    
    data[i]['has_osm_response'] = int(data[i]['osm_lat'] is not None)
    if data[i]['has_osm_response']:
        data[i]['osm_response_tlbr_match'] = int(is_tlbr_match(data[i]['osm_lat'], data[i]['osm_lat']))
        data[i]['osm_google_diff_meters'] = 1000 * points_distance(data[i]['google_lat'], data[i]['google_lon'], 
                                                                 data[i]['osm_lat'], data[i]['osm_lon'])

df = pd.DataFrame(data)
df.head()
df.to_excel('LOGDATA-1134.01.xlsx', index=None)

In [138]:
df['has_osm_response'].mean()

0.6326185101580135

In [140]:
for diff_threshold in [10, 20, 50, 100]:
    ddf = df[(df['has_osm_response'] == 1)]
    # print((df['google_yndx_diff_meters'] > diff_threshold).mean())
    print(diff_threshold, '%0.2f' % (100 * (ddf['osm_google_diff_meters'] > diff_threshold).mean()))
    # print()

10 85.82
20 69.22
50 51.47
100 40.59
